# Fuchsian Triangle Groups (Von Dyck groups)

We're going to implement a generator for the ordinary triangle group (or von Dyck group) $(p, q, r)$ for positive integers satisfying $\frac{1}{p} + \frac{1}{q} + \frac{1}{r} < 1$.

The group is generated by counter-clockwise rotations of angles $2\pi/p, 2\pi/q, 2\pi/r$ centered at the vertices of a triangle which (ordered in the counterclockwise sense) have interior angles of $\pi/p, \pi/q, \pi/r$ respectively.

To begin we calculate the length of the three sides.

In [3]:
from math import pi, cos, sin, acosh

def trianglesides(p,q,r):
	'''Returns the three sides of the triangle with interior angles pi/p,pi/q,pi/r.'''
	def secondlawofcosines(alpha,beta,gamma):
		'''Returns side opposite to angle gamma via horrible hyperbolic trigonometry.'''
		return acosh((cos(gamma)+cos(alpha)*cos(beta))/(sin(alpha)*sin(beta)))
	alpha,beta,gamma = pi/p, pi/q,pi/r
	return secondlawofcosines(beta, gamma, alpha), secondlawofcosines(gamma, alpha, beta), secondlawofcosines(alpha, beta, gamma)


The smallest triplet $(p,q,r)$ satisfying the required inequality is $(2,3,7)$ one can verify below that the smallest length is opposite to the smallest angle as it should be.

In [4]:
trianglesides(2,3,7)

(0.6206717375563866, 0.5452748317535436, 0.28312815336765745)

We're going to set things up so that a,b,c are the sides opposite to angles alpha, beta, and gamma respectively. And so that palpha, pbeta, pgamma are the vertices with angles alpha, beta and gamma respectively.  Then we will draw the result to see if it matches up well.

In [5]:
from dibujos import *
p, q, r = 2,3,7
a, b, c = trianglesides(p,q,r)
alpha, beta, gamma = pi/p, pi/q, pi/r
palpha,pbeta,pgamma = Point(0), Point.frompolar(radius=b,angle=0), Point.frompolar(radius=c,angle=alpha)

triangle = Figure([palpha,pbeta,pgamma,Segment(palpha,pbeta),Segment(pbeta,pgamma),Segment(pgamma,palpha)])

triangle.writesvg('1.svg')

![1.svg](1.svg)

So far so good.   Next we need the rotations of angles $2\alpha,2\beta,2\gamma$ around palpha, pbeta, and pgamma.   For this we need a tangent taking Point(0) to pbeta, and pgamma respectively.  And we use it to conjugate the rotations centered at Point(0).

In [6]:
topbeta, frompbeta = Tangent.rotate(alpha)*Tangent.forward(c), Tangent.forward(-c)*Tangent.rotate(-alpha)
topgamma, frompgamma = Tangent.forward(b), Tangent.forward(-b)
Ralpha, Rbeta, Rgamma = Tangent.rotate(2*alpha), topbeta*Tangent.rotate(2*beta)*frompbeta, topgamma*Tangent.rotate(2*gamma)*frompgamma

Let's check by generating a figure.

In [21]:
redtriangle = Figure(Ralpha*triangle)
for drawable in redtriangle:
    Red(drawable)

bluetriangle = Figure(Rbeta*triangle)
for drawable in bluetriangle:
    Blue(drawable)

greentriangle = Figure(Rgamma*triangle)
for drawable in greentriangle:
    Green(drawable)

g = Figure()
g.update(triangle)
g.update(redtriangle)
g.update(bluetriangle)
g.update(greentriangle)
g.update(Ralpha*bluetriangle)
g.update(Rgamma*Ralpha*bluetriangle)

g.writesvg('2.svg')

![2.svg](2.svg)

Now comes the hard part.  There are several relationships satisfied by the rotations.  The first, trivial one is that $R_\alpha^p = R_\beta^q = R_\gamma^r = 1$ where $1$ represents the identity transformation (we represent Ralpha by $R_\alpha$ here because it's nicer).

A less trivial relationship is that $R_\gamma R_\beta R_\alpha = 1$.  We should verify this.  Printing the product directly gives a messy complex matrix.  It's cleaner to just see what it does when applied to the origin and the boundary point at $1$.

In [8]:
print(Rgamma*Rbeta*Ralpha*Point(0))
print(Rgamma*Rbeta*Ralpha*Boundarypoint(0))

(1.051695859421949e-16-1.0345224291586447e-16j)
Boundarypoint(angle=-0.000)


Yup.  It works.

Ok, so here's the thing.  We want to generate all possible products of $R_\alpha, R_\beta, R_\gamma$ except we don't want to ever generate the same transformation twice.

To do this we consider the list of generators $R_\alpha, R_\alpha^{-1}, R_\beta, R_\beta^{-1}, R_\gamma, R_\gamma^{-1}$ (notice that in our example $R_{\alpha} = R_{\alpha}^{-1}$ so it appears twice in the list under different names, this annoying special case has to be checked out in what follows).

Next we try to generate all products of less than $n$ elements of the above "generating set" such that:
- No element is followed by its inverse.
- $R_\alpha$ never appears more than $\lfloor p/2\rfloor$ consecutive times.
- $R_\alpha^{-1}$ never appears more than $\lfloor (p-1)/2\rfloor$ consecutive times (here the case $p=2$ is dealt with).
- $R_\beta$ never appears more than $\lfloor q/2\rfloor$ consecutive times.
- $R_\beta^{-1}$ never appears more than $\lfloor (q-1)/2\rfloor$ consecutive times.
- $R_\gamma$ never appears more than $\lfloor r/2\rfloor$ consecutive times.
- $R_\gamma^{-1}$ never appears more than $\lfloor (r-1)/2\rfloor$ consecutive times.
- None of the "words" $R_\gamma R_\beta, R_\beta R_\alpha, R_\alpha R_\gamma$ appear in the product (notice that each one of them could be replaced by a single generator).
- None of the "words" $R_\beta^{-1} R_\gamma^{-1}, R_\alpha^{-1} R_\beta^{-1}, R_\gamma^{-1} R_\alpha^{-1}$ appear in the product (here one must be careful if some generator is equal to its inverse).

For lack of a better idea let's go with the following scheme.  We call the generators 'a','A','b','B',and 'c','C' so that 'a' is the inverse of 'A', etc.  Then we make a list of strings which are to be avoided by the above rules.  Then we generate all words which avoid those of the list incrementally (adding one letter at a time and verifying the rules are satisfied).


In [29]:
def trianglegroup(n,p=2,q=3,r=7):
    a, b, c = trianglesides(p,q,r)
    alpha, beta, gamma = pi/p, pi/q, pi/r
    palpha,pbeta,pgamma = Point(0), Point.frompolar(radius=b,angle=0), Point.frompolar(radius=c,angle=alpha)

    topbeta, frompbeta = Tangent.rotate(alpha)*Tangent.forward(c), Tangent.forward(-c)*Tangent.rotate(-alpha)
    topgamma, frompgamma = Tangent.forward(b), Tangent.forward(-b)
    Ralpha, Rbeta, Rgamma = Tangent.rotate(2*alpha), topbeta*Tangent.rotate(2*beta)*frompbeta, topgamma*Tangent.rotate(2*gamma)*frompgamma
    
    generators = {'a':Ralpha, 'A':Ralpha**(p-1), 'b':Rbeta, 'B':Rbeta**(q-1), 'c':Rgamma, 'C':Rgamma**(r-1)}

    avoid = 'aA Aa bB Bb cC Cc'.split()
    avoid.append('a'*(1+p//2))
    avoid.append('A'*(1+(p-1)//2))
    avoid.append('b'*(1+q//2))
    avoid.append('B'*(1+(q-1)//2))
    avoid.append('c'*(1+r//2))
    avoid.append('C'*(1+(r-1)//2))

    relations = 'cb ba ac BC AB CA'
    if p == 2:
        relations += ' '+relations.replace('A','a')
    if q == 2:
        relations += ' '+relations.replace('B','b')
    if r == 2:
        relations += ' '+relations.replace('C','c')
    
    avoid.extend(relations.split())
    
    def isgeodesic(word):
        for r in avoid:
            if r in word:
                return False
        return True
    
    geodesics = [['']]
    for i in range(1,n+1):
        newgeodesics = []
        for w in geodesics[i-1]:
            for l in 'aAbBcC':
                if isgeodesic(w+l):
                    newgeodesics.append(w+l)
        geodesics.append(newgeodesics)

    def wordtotangent(word):
        result = Tangent.origin()
        for letter in word:
            result = result*generators[letter]
        return result
                    
    for i in range(n+1):
        for w in geodesics[i]:
            yield wordtotangent(w)    

In [ ]:
p, q, r = 3, 3, 4
a, b, c = trianglesides(p,q,r)
alpha, beta, gamma = pi/p, pi/q, pi/r
palpha,pbeta,pgamma = Point(0), Point.frompolar(radius=b,angle=0), Point.frompolar(radius=c,angle=alpha)

triangle = Figure([palpha,pbeta,pgamma,Segment(palpha,pbeta),Segment(pbeta,pgamma),Segment(pgamma,palpha)])


g = Figure()
s = Tangent.rotate(0.4)*Tangent.forward(0.15)*stickman(size=0.1)
for t in trianglegroup(4,p=3,q=3,r=4):
    g.update(t*triangle)
    g.update(t*s)
g.writesvg('3.svg')
g.writepgf('3.pgf')

![3.svg](3.svg)